In [4]:
import os

# Define the target directory
target_directory = os.getcwd()  # change your directory here

# Check if the directory exists
if os.path.exists(target_directory):
    # Change the current working directory
    os.chdir(target_directory)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Directory {target_directory} does not exist.")

Directory changed to c:\Users\pablosal\Desktop\azure-ai-agent-services-demo


In [2]:
import pkg_resources
print(pkg_resources.get_distribution("synapseml").version)

0.11.0


C:\Users\pablosal\AppData\Local\Temp\ipykernel_38576\2673231978.py:1: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [ ]:
%pip install "openai==1.14.1"
%pip install httpx==0.27.2
%pip install synapseml==0.11.0
%pip install mlflow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached synapseml-0.11.0-py2.py3-none-any.whl.metadata (642 bytes)
Using cached synapseml-0.11.0-py2.py3-none-any.whl (514 kB)
  Attempting uninstall: synapseml
    Found existing installation: synapseml 1.0.10
    Uninstalling synapseml-1.0.10:
      Successfully uninstalled synapseml-1.0.10
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from synapse.ml. import *

ModuleNotFoundError: No module named 'synapse.ml.mlflow'

In [2]:
import requests
import json
import pprint
import typing as t
import time
import uuid

from openai import OpenAI
from openai._exceptions import APIStatusError
from openai._models import FinalRequestOptions
from openai._types import Omit
from openai._utils import is_given
from synapse.ml.mlflow import get_mlflow_env_config
from sempy.fabric._token_provider import SynapseTokenProvider
 
base_url = "https://api.fabric.microsoft.com/v1/workspaces/0633b854-4f74-4673-86a1-51ca4a8c1913/aiskills/9aa31dbe-acb4-4b76-8962-a1663759285f/aiassistant/openai"
question = "What datasources do you have access to?"

configs = get_mlflow_env_config()

# Create OpenAI Client
class FabricOpenAI(OpenAI):
    def __init__(
        self,
        api_version: str ="2024-05-01-preview",
        **kwargs: t.Any,
    ) -> None:
        self.api_version = api_version
        default_query = kwargs.pop("default_query", {})
        default_query["api-version"] = self.api_version
        super().__init__(
            api_key="",
            base_url=base_url,
            default_query=default_query,
            **kwargs,
        )
    
    def _prepare_options(self, options: FinalRequestOptions) -> None:
        headers: dict[str, str | Omit] = (
            {**options.headers} if is_given(options.headers) else {}
        )
        options.headers = headers
        headers["Authorization"] = f"Bearer {configs.driver_aad_token}"
        if "Accept" not in headers:
            headers["Accept"] = "application/json"
        if "ActivityId" not in headers:
            correlation_id = str(uuid.uuid4())
            headers["ActivityId"] = correlation_id

        return super()._prepare_options(options)

# Pretty printing helper
def pretty_print(messages):
    print("---Conversation---")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()

fabric_client = FabricOpenAI()
# Create assistant
assistant = fabric_client.beta.assistants.create(model="not used")
# Create thread
thread = fabric_client.beta.threads.create()
# Create message on thread
message = fabric_client.beta.threads.messages.create(thread_id=thread.id, role="user", content=question)
# Create run
run = fabric_client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant.id)

# Wait for run to complete
while run.status == "queued" or run.status == "in_progress":
    run = fabric_client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id,
    )
    print(run.status)
    time.sleep(2)

# Print messages
response = fabric_client.beta.threads.messages.list(thread_id=thread.id, order="asc")
pretty_print(response)

# Delete thread
fabric_client.beta.threads.delete(thread_id=thread.id)

ModuleNotFoundError: No module named 'synapse.ml.mlflow'

In [4]:
!pip install synapseml


In [1]:
import synapse.ml
print(synapse.ml.__version__)

AttributeError: module 'synapse.ml' has no attribute '__version__'